# **Downloading The dataset**

you need to download the images from the dataset,The excel file provided in the labeled dataset contains the images paths and thier labels. make sure that the images are downloaded into the following directory (/content/Data)

# **Data preprocessing and augmentation**

The excel file provided in the labeled dataset contains the images paths and thier labels.

In [ ]:
cd /content/Data

In [ ]:
import os
import pandas as pd
from shutil import copyfile

def organize_images_by_label(excel_path, image_column, label_column, output_dir):
    # Read the Excel file
    data = pd.read_excel(excel_path)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through rows and copy images to label-specific folders
    for index, row in data.iterrows():
        try:
            image_path = str(row[image_column])  # Convert to string
            label = str(row[label_column])  # Convert to string

            # Skip rows where image path or label is not a valid string
            if not image_path.strip() or not label.strip():
                print(f"Skipping row {index + 2} due to empty image path or label.")
                continue

            # Create label folder if it doesn't exist
            label_folder = os.path.join(output_dir, label)
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)

            # Copy image to label folder
            _, image_filename = os.path.split(image_path)
            destination_path = os.path.join(label_folder, image_filename)

            # Check if destination image already exists, and if not, copy
            if not os.path.exists(destination_path):
                copyfile(image_path, destination_path)
            else:
                print(f"Warning: Image '{image_filename}' already exists in '{label}' folder.")
        except Exception as e:
            print(f"Error processing row {index + 2}: {e}")

if __name__ == "__main__":
    # Replace these with your actual file paths and column names
    excel_file_path = '/content/Labels.xlsx'
    image_column_name = 'image_path'
    label_column_name = 'Best_DET'
    output_directory = '/content/new_data/data'

    organize_images_by_label(excel_file_path, image_column_name, label_column_name, output_directory)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data'

    count_images_in_folders(root_folder_path)


**The following cells should be run 8 times, one time for each class to generate more data samples. just change the name of the detector in the path.**

for example:
change ("/content/new_data/data/FTRCNN") to ("/content/new_data/data/DETR") and so on for each class.

In [ ]:
pip install Augmentor

In [ ]:
import os
from PIL import Image

def count_images_in_folder(folder_path):
    # Ensure the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return

    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg'))]

    # Count the number of image files
    num_images = len(image_files)

    print(f"Number of images in folder '{folder_path}': {num_images}")

# Example usage:
folder_path = '/content/new_data/data/FTRCNN'
count_images_in_folder(folder_path)


In [ ]:
import Augmentor
p = Augmentor.Pipeline("/content/new_data/data/FTRCNN")

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)

In [ ]:
p.sample(2000)

In [ ]:
import os
import shutil

def move_images(source_folder, destination_folder):
    # Ensure both source and destination folders exist
    if not os.path.exists(source_folder):
        print(f"Error: Source folder '{source_folder}' not found.")
        return

    if not os.path.exists(destination_folder):
        print(f"Error: Destination folder '{destination_folder}' not found.")
        return

    # Get a list of all files in the source folder
    all_files = os.listdir(source_folder)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg','.png'))]

    # Move each image file to the destination folder
    for image_file in image_files:
        source_path = os.path.join(source_folder, image_file)
        destination_path = os.path.join(destination_folder, image_file)

        # Move the file
        shutil.move(source_path, destination_path)

        print(f"Moved: {image_file}")

    print("Image move operation completed.")

# Example usage:
source_folder = '/content/new_data/data/FTRCNN/output'
destination_folder = '/content/new_data/data/FTRCNN'

move_images(source_folder, destination_folder)


In [ ]:
import os
from PIL import Image

def count_images_in_folder(folder_path):
    # Ensure the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return

    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg','png'))]

    # Count the number of image files
    num_images = len(image_files)

    print(f"Number of images in folder '{folder_path}': {num_images}")

# Example usage:
folder_path = '/content/new_data/data/FTRCNN'
count_images_in_folder(folder_path)


**The following cells will split the data into training and testing splits to prepare for the training**

In [ ]:
import os
from sklearn.model_selection import train_test_split
from shutil import copyfile

def split_dataset(input_folder, output_train_folder, output_test_folder, test_size=0.2, random_seed=42):
    # Get the list of class folders in the input folder
    class_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]

    # Create output folders if they don't exist
    os.makedirs(output_train_folder, exist_ok=True)
    os.makedirs(output_test_folder, exist_ok=True)

    # Loop through each class folder
    for class_folder in class_folders:
        class_path = os.path.join(input_folder, class_folder)

        # Get the list of image files for the current class
        image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg') or f.endswith('.png')]

        # Check if there are enough samples for splitting
        if len(image_files) < 2:
            print(f"Skipping class '{class_folder}' due to insufficient samples.")
            continue

        # Split the image files into training and testing sets
        train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=random_seed)

        # Copy training images to the output training folder
        for train_file in train_files:
            input_path = os.path.join(class_path, train_file)
            output_path = os.path.join(output_train_folder, class_folder, train_file)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            copyfile(input_path, output_path)

        # Copy testing images to the output testing folder
        for test_file in test_files:
            input_path = os.path.join(class_path, test_file)
            output_path = os.path.join(output_test_folder, class_folder, test_file)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            copyfile(input_path, output_path)

    print("Dataset split into training and testing sets.")

# Example usage:
input_folder = '/content/new_data/data'
output_train_folder = '/content/new_data/data/train'
output_test_folder = '/content/new_data/data/valid'

split_dataset(input_folder, output_train_folder, output_test_folder)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data/train'

    count_images_in_folders(root_folder_path)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data/valid'

    count_images_in_folders(root_folder_path)


**Training the Vi-T model**

In [ ]:
pip install timm

In [ ]:
# -*- coding: utf-8 -*-
"""ViT+CE

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1_ijz8987KSV9W-5z4Ux0YqQBSmgeywc2
"""

!pip install timm

#!unzip "/content/drive/MyDrive/ViT_Data.zip" -d "/content/drive/MyDrive/ViT_Data"

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

DATA_PATH = '/content/new_data/data'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(DATA_PATH + '/train', transform=transform)
val_dataset = datasets.ImageFolder(DATA_PATH + '/valid', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64 )

import timm

class VisionTransformerClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super(VisionTransformerClassification, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = torch.nn.Linear(self.vit.head.in_features, num_classes)

    def forward(self, x):
        return self.vit(x)

NUM_CLASSES = len(train_dataset.classes)
model = VisionTransformerClassification(NUM_CLASSES)

import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct_train / total_train
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f'Validation accuracy: {100. * correct / total:.2f}%')